In [14]:
!pip install -U langchain-google-genai pinecone-client python-dotenv



Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 20.0 MB/s  0:00:00
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -------------------------- ------------- 3.1/4.7 MB 16.3 MB/s eta 0:00:01
   ------------------------------------- -- 4.5/4.7 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 4.7/4.7 MB 10.5 MB/s  0:00:00

   ----------------------------------------  0/13 [pyasn1]
   ----------------------------------------  0/13 [pyasn1]
   ----------------------------------------  0/13 [pyasn1]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

assert os.getenv("GOOGLE_API_KEY"), "Falta GOOGLE_API_KEY en .env"
assert os.getenv("PINECONE_API_KEY"), "Falta PINECONE_API_KEY en .env"

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "rag-demo-index")



In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "rag-demo-index"


if pc.has_index(index_name):
    pc.delete_index(index_name)


pc.create_index(
    name=index_name,
    dimension=3072, 
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)
print("índice creado con dimensión 3072")



✅ Nuevo índice creado con dimensión 3072


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GOOGLE_API_KEY)
print("Embeddings de Gemini")



✅ Embeddings de Gemini inicializados


In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

✅ Vector store listo para indexar documentos


In [ ]:
from langchain_core.documents import Document
from uuid import uuid4

documents = [
    Document(page_content="LangChain simplifies building LLM apps.", metadata={"source": "tutorial"}),
    Document(page_content="Gemini by Google provides generative AI models.", metadata={"source": "news"}),
]

ids = [str(uuid4()) for _ in documents]
vector_store.add_documents(documents=documents, ids=ids)




✅ Documentos añadidos al vector store


In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 2, "score_threshold": 0.4},
)

✅ Retriever configurado correctamente


In [33]:
query = "What is Gemini?"
results = vector_store.similarity_search(query, k=2)
print("🔍 Results:")
for r in results:
    print(r.page_content, r.metadata)



🔍 Results:
Gemini by Google provides generative AI models. {'source': 'news'}
Gemini by Google provides generative AI models. {'source': 'news'}


In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY, temperature=0)

prompt = ChatPromptTemplate.from_template(
    "Use the following context to answer the question.\n\n{context}\n\nQ: {input}"
)

# Recuperar contexto
contexts = [r.page_content for r in results]
combined_context = "\n".join(contexts)

response = llm.invoke([
    {"role":"system", "content":"You are a helpful assistant."},
    {"role":"user", "content": prompt.format(context=combined_context, input=query)}
])
print("🤖 Answer:\n", response.content)


🤖 Answer:
 Gemini is a generative AI model provided by Google.
